In [1]:
import os
os.chdir('/home/garskova-d/metrika-analytics')

from metr_utils.databases import clickhouse as ch
from metr_utils.databases import yql_db
from metr_utils.databases import yt_db
from metr_utils.databases import mysql_db
from metr_utils.databases import psql_db
from metr_utils.configs import options
from metr_utils.configs import get_vault_data
from metr_utils import generate_query
from metr_utils import plotly_graphs as pplot
from metr_utils import pandas_helpers as ph
from metr_utils import blackbox as bb
import pandas as pd
import numpy as np
from io import StringIO
from datetime import datetime, timedelta
import time
import builtins
builtins.opts = options.args(get_robot_passes=True)

In [2]:
import getpass
import vault_client
bot_name = 'robot-conv-main'
yav = vault_client.instances.Production()
# Версии нужно обновлять руками
passes = yav.get_version('ver-01f6q4fsg0t82b3byr22ametqs')['value']  # Все основные пароли
passes.update(yav.get_version('ver-01g6x0fd2ptb1r684nezh88zdj')['value']) 
password = passes['admin']
query = '''
show tables in click_storage
FORMAT TabSeparatedWithNames
'''
df_new = ch.get_df(query=query,host = 'https://sas-7vnn9nqd22rekwsp.db.yandex.net:8443', 
                               user = 'admin', 
                               password = password, 
                               database = 'click_storage')
df_new

,name
0,market_click_storage


In [69]:
import getpass
import vault_client
bot_name = 'robot-conv-main'
yav = vault_client.instances.Production()
# Версии нужно обновлять руками
passes = yav.get_version('ver-01f6q4fsg0t82b3byr22ametqs')['value']  # Все основные пароли
passes.update(yav.get_version('ver-01g6x0fd2ptb1r684nezh88zdj')['value']) 
password = passes['admin']
query = '''
desc market_click_storage
FORMAT TabSeparatedWithNames
'''
df_new = ch.get_df(query=query,host = 'https://sas-7vnn9nqd22rekwsp.db.yandex.net:8443', 
                               user = 'admin', 
                               password = password, 
                               database = 'click_storage')
df_new

,name,type,default_type,default_expression,comment,codec_expression,ttl_expression
0,Date,Date,NaN,NaN,NaN,NaN,NaN
1,Sign,Int8,NaN,NaN,NaN,NaN,NaN
2,ShopID,UInt64,NaN,NaN,NaN,NaN,NaN
3,RowIDHash,UInt64,NaN,NaN,NaN,NaN,NaN
4,RowID,String,NaN,NaN,NaN,NaN,NaN
5,UserID,UInt64,NaN,NaN,NaN,NaN,NaN
6,EventTime,DateTime,NaN,NaN,NaN,NaN,NaN
7,UTMSource,String,NaN,NaN,NaN,NaN,NaN
8,UTMMedium,String,NaN,NaN,NaN,NaN,NaN
9,UTMCampaign,String,NaN,NaN,NaN,NaN,NaN


In [10]:
query = '''
desc market_click_storage
FORMAT TabSeparatedWithNames
'''
df_old = ch.get_df(query=query, host='http://mtgiga001-1t.metrika.yandex.net:8123', user = 'default', override_default = True)
df_old

,name,type,default_type,default_expression,comment,codec_expression,ttl_expression
0,Date,Date,NaN,NaN,NaN,NaN,NaN
1,Sign,Int8,NaN,NaN,NaN,NaN,NaN
2,ShopID,UInt64,NaN,NaN,NaN,NaN,NaN
3,RowIDHash,UInt64,NaN,NaN,NaN,NaN,NaN
4,RowID,String,NaN,NaN,NaN,NaN,NaN
5,UserID,UInt64,NaN,NaN,NaN,NaN,NaN
6,EventTime,DateTime,NaN,NaN,NaN,NaN,NaN
7,UTMSource,String,NaN,NaN,NaN,NaN,NaN
8,UTMMedium,String,NaN,NaN,NaN,NaN,NaN
9,UTMCampaign,String,NaN,NaN,NaN,NaN,NaN


### По датам

In [70]:
import getpass
import vault_client
bot_name = 'robot-conv-main'
yav = vault_client.instances.Production()
# Версии нужно обновлять руками
passes = yav.get_version('ver-01f6q4fsg0t82b3byr22ametqs')['value']  # Все основные пароли
passes.update(yav.get_version('ver-01g6x0fd2ptb1r684nezh88zdj')['value']) 
password = passes['admin']
query = '''
select toStartOfMonth(Date) as month, sum(Sign) as clicks
from market_click_storage
group by month
FORMAT TabSeparatedWithNames
'''
df_new = ch.get_df(query=query,host = 'https://sas-7vnn9nqd22rekwsp.db.yandex.net:8443', 
                               user = 'admin', 
                               password = password, 
                               database = 'click_storage')
df_new

,month,clicks
0,2021-02-01,16465054
1,2021-03-01,129515233
2,2021-04-01,70964619
3,2021-05-01,59290320
4,2021-06-01,37870588
5,2021-07-01,42710090
6,2021-08-01,41517224
7,2021-09-01,36017388
8,2021-10-01,35963574
9,2021-11-01,38609055


In [12]:
query = '''
select toStartOfMonth(Date) as month, sum(Sign) as clicks
from market_click_storage
group by month
FORMAT TabSeparatedWithNames
'''
df_old = ch.get_df(query=query, host='http://mtgiga001-1t.metrika.yandex.net:8123', user = 'default', override_default = True)
df_old

,month,clicks
0,2021-02-01,16465054
1,2021-03-01,129515233
2,2021-04-01,70964619
3,2021-05-01,59290320
4,2021-06-01,37870588
5,2021-07-01,42710090
6,2021-08-01,41517224
7,2021-09-01,36017388
8,2021-10-01,35963574
9,2021-11-01,38609055


In [14]:
df_comp_days = df_new.merge(df_old, how = 'outer', left_on = 'month', right_on = 'month', suffixes = ['_new','_old'])

In [17]:
df_comp_days[df_comp_days['clicks_new']!=df_comp_days['clicks_old']].shape[0]

0

In [18]:
df_comp_days

,month,clicks_new,clicks_old
0,2021-02-01,16465054,16465054
1,2021-03-01,129515233,129515233
2,2021-04-01,70964619,70964619
3,2021-05-01,59290320,59290320
4,2021-06-01,37870588,37870588
5,2021-07-01,42710090,42710090
6,2021-08-01,41517224,41517224
7,2021-09-01,36017388,36017388
8,2021-10-01,35963574,35963574
9,2021-11-01,38609055,38609055


#### проверяем выборочно

In [25]:
query = '''
SELECT *
FROM market_click_storage final
where Date >= '2022-05-01'
FORMAT TabSeparatedWithNames
'''
df_new_sample = ch.get_df(query=query,host = 'https://sas-7vnn9nqd22rekwsp.db.yandex.net:8443', 
                               user = 'admin', 
                               password = password, 
                               database = 'click_storage')
df_new_sample

KeyboardInterrupt: 

In [18]:
# %% time 

# query = '''
# SELECT *
# FROM click_storage_all final
# sample 0.00001
# where Date = '2022-04-11'
# FORMAT TabSeparatedWithNames
# '''
# df_old_sample = ch.get_df(query=query, host='http://mtgiga001-1t.metrika.yandex.net:8123', user = 'default', override_default = True)
# df_old_sample

,Date,Sign,UserID,RegionID,Age,Sex,Income,Interests,Robotness,LogID,...,RealDomainID,TemplateID,TopAncestorMD5,IsActionOnly,ActionCost,ActionCostCur,ActionCostTaxFree,UniversalColumn,Yhid,EstimatedClickTime
0,2022-04-11,1,1219315781638786149,10298,35,2,4,42,0,1847631165887930751,...,0,631,0,0,0,0,0,\b����������J\tundefined�\0\0\0\0\0\0�?��...,0,2022-04-11 16:49:09
1,2022-04-11,1,9954346181622136155,10309,25,2,7,8448,0,1845267217035885627,...,26495292,320,6328164764924076282,0,0,0,0,\b����ɣ�������ɣ�������ɣ��� ��������3J\tun...,0,2022-04-11 14:18:51
2,2022-04-11,1,2029342601644067792,10174,35,1,7,13183,0,1852049051506762302,...,21628792,410,0,0,0,0,0,\bЗ����� З����� Ɯ���׮��J\tundefinedx���\b�...,0,2022-04-11 21:30:02
3,2022-04-11,1,6787055511647962868,162,17,1,0,0,0,1840540814023913330,...,1253533,320,16868181572069635360,0,0,0,0,\b������^������^�������SJ\tcommander�\0\0\0...,0,2022-04-11 09:18:21
4,2022-04-11,1,6787055511647962868,162,17,1,0,0,0,1840543098851618965,...,1253533,320,16868181572069635360,0,0,0,0,\b������^������^�������SJ\tcommander�\0\0\0...,0,2022-04-11 09:18:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533,2022-04-11,1,3541693461646208572,4,0,0,0,0,0,1848312419350806365,...,26204066,320,8550650752760847894,0,0,0,0,\b����1��Ӳ����W��Ӳ����WJuac�\0\0\0\0\0\0�...,0,2022-04-11 17:32:28
534,2022-04-11,1,3541693461646208572,4,0,0,0,0,0,1848320909443399517,...,26204066,320,8550650752760847894,0,0,0,0,\b����1��Ӳ����W��Ӳ����WJuac�\0\0\0\0\0\0�...,0,2022-04-11 17:33:00
535,2022-04-11,1,3541693461646208572,4,0,0,0,0,0,1848323031310860344,...,26204066,320,8550650752760847894,0,0,0,0,\b����1��Ӳ����W��Ӳ����WJuac�\0\0\0\0\0\0�...,0,2022-04-11 17:33:08
536,2022-04-11,1,3541693461646208572,4,0,0,0,0,0,1848324912746135518,...,26204066,320,8550650752760847894,0,0,0,0,\b����1��Ӳ����W��Ӳ����WJuac�\0\0\0\0\0\0�...,0,2022-04-11 17:33:15


In [71]:
# df_sample = df_old_sample.merge(df_new_sample, how = 'outer', left_on = 'LogID', right_on = 'LogID', suffixes = ['_old','_new'], how = 'outer')

In [93]:
# set(df_old_sample.columns) - set(df_new_sample.columns)

{'PhraseProcessedID'}

In [94]:
# set(df_new_sample.columns) - set(df_old_sample.columns)

{'ActualFormat',
 'AutobudgetExperimentTrafficShare',
 'ContentTargetingsMD5',
 'HitUserAgentMD5',
 'IsDirectBannerDomainLoaded',
 'TrackID'}

In [88]:
# absent = set(df_new_sample.columns) - set(df_old_sample.columns)
# absent.update(set(df_old_sample.columns) - set(df_new_sample.columns))
# absent_list = list(absent)

In [96]:
# for el in df_new_sample.columns:
#     if el not in absent_list and el != 'LogID':
#         df_sample[str(el) + '_diff'] = df_sample[str(el) + '_old'] == df_sample[str(el) + '_new']
        
#         if df_sample[str(el) + '_diff'].sum() < df_sample[~df_sample[str(el) + '_old'].isna()].shape[0]:
#             print(str(el), df_sample[str(el) + '_diff'].sum())
#             print(df_sample[~df_sample[str(el) + '_diff'].isna()].shape[0])

UserID 537
538


In [95]:
#df_sample.loc[0, 'UserID_old'] = 1219315781638786148

#### проверяем выборочно - способ побыстрее

In [8]:
query = '''
desc market_click_storage
FORMAT TabSeparatedWithNames
'''
df_new_columns = ch.get_df(query=query,host = 'https://sas-7vnn9nqd22rekwsp.db.yandex.net:8443', 
                               user = 'admin', 
                               password = password, 
                               database = 'click_storage')
df_new_columns

,name,type,default_type,default_expression,comment,codec_expression,ttl_expression
0,Date,Date,NaN,NaN,NaN,NaN,NaN
1,Sign,Int8,NaN,NaN,NaN,NaN,NaN
2,ShopID,UInt64,NaN,NaN,NaN,NaN,NaN
3,RowIDHash,UInt64,NaN,NaN,NaN,NaN,NaN
4,RowID,String,NaN,NaN,NaN,NaN,NaN
5,UserID,UInt64,NaN,NaN,NaN,NaN,NaN
6,EventTime,DateTime,NaN,NaN,NaN,NaN,NaN
7,UTMSource,String,NaN,NaN,NaN,NaN,NaN
8,UTMMedium,String,NaN,NaN,NaN,NaN,NaN
9,UTMCampaign,String,NaN,NaN,NaN,NaN,NaN


In [36]:
query = '''
desc market_click_storage
FORMAT TabSeparatedWithNames
'''
df_old_columns = ch.get_df(query=query, host='http://mtgiga001-1t.metrika.yandex.net:8123', user = 'default', override_default = True)
df_old_columns

,name,type,default_type,default_expression,comment,codec_expression,ttl_expression
0,Date,Date,NaN,NaN,NaN,NaN,NaN
1,Sign,Int8,NaN,NaN,NaN,NaN,NaN
2,ShopID,UInt64,NaN,NaN,NaN,NaN,NaN
3,RowIDHash,UInt64,NaN,NaN,NaN,NaN,NaN
4,RowID,String,NaN,NaN,NaN,NaN,NaN
5,UserID,UInt64,NaN,NaN,NaN,NaN,NaN
6,EventTime,DateTime,NaN,NaN,NaN,NaN,NaN
7,UTMSource,String,NaN,NaN,NaN,NaN,NaN
8,UTMMedium,String,NaN,NaN,NaN,NaN,NaN
9,UTMCampaign,String,NaN,NaN,NaN,NaN,NaN


In [37]:
set(df_new_columns.name.values) - set(df_old_columns.name.values)

set()

In [38]:
set(df_old_columns.name.values) - set(df_new_columns.name.values)

set()

In [3]:
# absent = set(df_new_columns.name.values) - set(df_old_columns.name.values)
# absent.update(set(df_old_columns.name.values) - set(df_new_columns.name.values))
# absent_list = list(absent)
absent_list = []

In [4]:
absent_list

[]

In [9]:
col_names = list(df_new_columns.name.values) #choose

In [6]:
import tqdm

In [10]:
dict_compare = dict()
bad_elements = []

for param_name in tqdm.tqdm(col_names):
    
    if param_name != 'RowID' and param_name not in absent_list:
    
        print(param_name)

        query = f'''
        SELECT {param_name}, RowID, sum(Sign) as clicks
        FROM market_click_storage final
        sample 0.01
        where Date >= '2022-06-01' and Date <= '2022-07-04'
        group by {param_name}, RowID
        FORMAT TabSeparatedWithNames
        '''
        df_old = ch.get_df(query=query, host='http://mtgiga001-1t.metrika.yandex.net:8123', user = 'default', override_default = True)
        

        query = f'''
        select {param_name}, RowID, sum(Sign) as clicks
        FROM market_click_storage final
        sample 0.01
        where Date >= '2022-06-01' and Date <= '2022-07-04'
        group by {param_name}, RowID
        FORMAT TabSeparatedWithNames
        '''
        df_new = ch.get_df(query=query,host = 'https://sas-7vnn9nqd22rekwsp.db.yandex.net:8443', 
                               user = 'admin', 
                               password = password, 
                               database = 'click_storage')
        df_by_param = df_old.merge(df_new, how = 'outer', left_on = 'RowID', right_on = 'RowID', suffixes = ['_old','_new'])
        
        df_by_param[f'{param_name}' + '_diff'] = df_by_param[f'{param_name}' + '_old'] == df_by_param[f'{param_name}' + '_new']
        
        if df_by_param[f'{param_name}' + '_diff'].sum() < df_by_param[~df_by_param[f'{param_name}' + '_old'].isna()].shape[0]:
            print(f'{param_name}', df_by_param[f'{param_name}' + '_diff'].sum())
            print(df_by_param[~df_by_param[f'{param_name}' + '_diff'].isna()].shape[0])
            bad_elements.append(f'{param_name}')
            dict_compare[f'{param_name}'] = df_by_param

  0%|          | 0/16 [00:00<?, ?it/s]

Date


  6%|▋         | 1/16 [00:01<00:29,  1.99s/it]

Sign


  6%|▋         | 1/16 [00:03<00:54,  3.65s/it]


KeyboardInterrupt: 

In [95]:
dict_compare

{}

In [87]:
df_diff = dict_compare['Sign']

In [5]:
dict_compare = dict()
bad_elements = []

for param_name in tqdm.tqdm(col_names):
    
    if param_name != 'RowID' and param_name not in absent_list:
    
        print(param_name)

        query = f'''
        SELECT {param_name}, RowID, sum(Sign) as clicks
        FROM market_click_storage final
        sample 0.00001
        where Date >= '2022-01-01' and Date <= '2022-07-04'
        group by {param_name}, RowID
        FORMAT TabSeparatedWithNames
        '''
        df_old = ch.get_df(query=query, host='http://mtgiga001-1t.metrika.yandex.net:8123', user = 'default', override_default = True)
        

        query = f'''
        select {param_name}, RowID, sum(Sign) as clicks
        FROM market_click_storage final
        sample 0.00001
        where Date >= '2022-01-01' and Date <= '2022-07-04'
        group by {param_name}, RowID
        FORMAT TabSeparatedWithNames
        '''
        df_new = ch.get_df(query=query,host = 'https://sas-7vnn9nqd22rekwsp.db.yandex.net:8443', 
                               user = 'admin', 
                               password = password, 
                               database = 'click_storage')
        df_by_param = df_old.merge(df_new, how = 'outer', left_on = 'RowID', right_on = 'RowID', suffixes = ['_old','_new'])
        
        df_by_param[f'{param_name}' + '_diff'] = df_by_param[f'{param_name}' + '_old'] == df_by_param[f'{param_name}' + '_new']
        
        if df_by_param[f'{param_name}' + '_diff'].sum() < df_by_param[~df_by_param[f'{param_name}' + '_old'].isna()].shape[0]:
            print(f'{param_name}', df_by_param[f'{param_name}' + '_diff'].sum())
            print(df_by_param[~df_by_param[f'{param_name}' + '_diff'].isna()].shape[0])
            bad_elements.append(f'{param_name}')
            dict_compare[f'{param_name}'] = df_by_param

NameError: name 'tqdm' is not defined

In [97]:
dict_compare.keys()

dict_keys([])

In [161]:
# diff_good_event = dict_compare['GoodEvent']
# print(diff_good_event[diff_good_event['GoodEvent_diff']==False].shape[0])
# diff_good_event[diff_good_event['GoodEvent_diff']==False]

2842


,GoodEvent_old,LogID,clicks_old,GoodEvent_new,clicks_new,GoodEvent_diff
92,-8,1977770123070329644,1,1,0,False
262,1,1995142033687768709,0,-8,1,False
263,-8,1995142033687768709,1,1,0,False
327,-8,1978571369796129884,1,1,0,False
328,1,1978571369796129884,0,-8,1,False
...,...,...,...,...,...,...
915048,-8,1977371732680434035,1,1,0,False
916281,-8,1993545981966996737,1,1,0,False
917356,-8,1977677587827458290,1,1,0,False
921597,-8,1978083716938137858,1,1,0,False


In [166]:
# diff_UniversalColumn = dict_compare['UniversalColumn']
# print(diff_UniversalColumn[diff_UniversalColumn['UniversalColumn_diff']==False].shape[0])
# diff_UniversalColumn[diff_UniversalColumn['UniversalColumn_diff']==False]

1


,UniversalColumn_old,LogID,clicks_old,UniversalColumn_new,clicks_new,UniversalColumn_diff
445627,����Ɵ���J\tundefinedj$394101de-7d3f-451b-b8a...,1993106600600598021,1,����Ɵ���J\tundefinedj$394101de-7d3f-451b-b8a9...,1,False


#### проверяем интегрально

In [54]:
for el in df_new_columns.name.values:
    print(el)

Date
Sign
ShopID
RowIDHash
RowID
UserID
EventTime
UTMSource
UTMMedium
UTMCampaign
UTMContent
UTMTerm
Price
ShowUid
IsRollback
State


In [9]:
important_params = ['RowIDHash', 'UTMSource', 'UTMMedium', 'UTMCampaign',
                   'IsRollback', 'State', 'ShowUid']

In [10]:
numeric_metrics = [
'Price'
]

In [11]:
heavy_columns = []

In [12]:
heavy_columns

[]

In [13]:
import tqdm

In [17]:
bad_integral_metrics = []
bad_integral_metrics_dict = dict()
aggregated_metrics = dict()
for metric_name in tqdm.tqdm(numeric_metrics):
    
    print(metric_name)
    
    if metric_name not in bad_integral_metrics_dict.keys():
        bad_integral_metrics_dict[metric_name] = dict()
    if metric_name not in aggregated_metrics.keys():
        aggregated_metrics[metric_name] = dict()
    
    for param_name in important_params:

        if param_name not in ('RowID', 'UserID', 'ShopID') and (param_name not in absent_list):
            
            if param_name not in aggregated_metrics[metric_name].keys() and param_name not in heavy_columns and param_name != metric_name:
                
                print(param_name)
                aggregated_metrics[metric_name][param_name] = ''

                query = f'''
                SELECT ShopID, RowID, UserID, {param_name}, sum(Sign * {metric_name}) as {metric_name}
                FROM market_click_storage
                sample 0.01
                where Date >= '2022-07-04' and Date <= '2022-07-04'
                group by ShopID, RowID, UserID, {param_name}
                FORMAT TabSeparatedWithNames
                '''
                df_old = ch.get_df(query=query, host='http://mtgiga001-1t.metrika.yandex.net:8123', user = 'default', override_default = True)


                query = f'''
                select ShopID, RowID, UserID, {param_name}, sum(Sign * {metric_name}) as {metric_name}
                FROM market_click_storage
                sample 0.01
                where Date >= '2022-07-04' and Date <= '2022-07-04'
                group by ShopID, RowID, UserID, {param_name} 
                FORMAT TabSeparatedWithNames
                '''
                df_new = ch.get_df(query=query,host = 'https://sas-7vnn9nqd22rekwsp.db.yandex.net:8443', 
                               user = 'admin', 
                               password = password, 
                               database = 'click_storage')
                df_by_param = df_old.merge(df_new, how = 'outer', left_on = f'{param_name}', right_on = f'{param_name}', suffixes = ['_old','_new'])

                df_by_param[f'{metric_name}' + '_diff'] = df_by_param[f'{metric_name}' + '_old'] == df_by_param[f'{metric_name}' + '_new']

                if df_by_param[f'{metric_name}' + '_diff'].sum() < df_by_param.shape[0]:
                    print(f'{metric_name}',f'{param_name}', df_by_param[f'{metric_name}' + '_diff'].sum())
                    print(df_by_param[~df_by_param[f'{metric_name}' + '_old'].isna()].shape[0])
                    bad_integral_metrics.append((f'{metric_name}', f'{param_name}'))
                    bad_integral_metrics_dict[metric_name][param_name] = df_by_param
                

  0%|          | 0/1 [00:00<?, ?it/s]

Price
RowIDHash
Price RowIDHash 1145
1152
UTMSource
Price UTMSource 20895
661729
UTMMedium
Price UTMMedium 25299
795118
UTMCampaign
Price UTMCampaign 25083
813213
IsRollback
Price IsRollback 40914
1319047
State
Price State 40891
1319040
ShowUid


100%|██████████| 1/1 [00:02<00:00,  2.19s/it]

Price ShowUid 1145
1152


In [18]:
bad_integral_metrics_dict.keys()

dict_keys(['Price'])

In [19]:
bad_integral_metrics_dict['Price'].keys()

dict_keys(['RowIDHash', 'UTMSource', 'UTMMedium', 'UTMCampaign', 'IsRollback', 'State', 'ShowUid'])

In [20]:
df_diff = bad_integral_metrics_dict['Price']['RowIDHash']

In [21]:
df_diff[df_diff['Price_diff']!=True]

,ShopID_old,RowID_old,UserID_old,RowIDHash,Price_old,ShopID_new,RowID_new,UserID_new,Price_new,Price_diff
467,38966,SasMarketClickdRedirLog:241744531:0:6,62911081614845583,8306921881893493526,0,NaN,NaN,NaN,NaN,False
538,38966,VlaMarketClickdRedirLog:453275085:0:9,62911081614845583,3183945062465840003,0,NaN,NaN,NaN,NaN,False
566,38966,IvaMarketClickdRedirLog:363804484:0:6,62911081614845583,987934137377584104,0,NaN,NaN,NaN,NaN,False
648,38966,VlaMarketClickdRedirLog:453284664:0:6,62911081614845583,15558555536925139894,0,NaN,NaN,NaN,NaN,False
696,38966,IvaMarketClickdRedirLog:363800699:0:2,62911081614845583,2740347147268533724,0,NaN,NaN,NaN,NaN,False
703,38966,VlaMarketClickdRedirLog:453228196:0:2,62911081614845583,9693392955598754670,0,NaN,NaN,NaN,NaN,False
1122,38966,VlaMarketClickdRedirLog:453241289:0:6,62911081614845583,712614380363318546,0,NaN,NaN,NaN,NaN,False


In [181]:
row_id_strange = df_diff[df_diff['Price_diff']!=True].RowIDHash

AttributeError: 'DataFrame' object has no attribute 'RowIDHash'

In [168]:
row_id_strange = set(row_id_strange)

In [178]:
row_id_strange

{482083832131146747,
 712614380363318546,
 987934137377584104,
 2740347147268533724,
 3183945062465840003,
 4899026660956566000,
 5731425724385502725,
 6936617777469473307,
 7032604313754320162,
 8306921881893493526,
 9693392955598754670,
 10339061786092694014,
 10556636132099394870,
 12314582487736528525,
 12767468915219930199,
 12858429085624914644,
 14550371238004333438,
 15051863895377819635,
 15558555536925139894,
 15917038717828998480,
 15945199290751418947,
 16136896081935896271,
 17912970360559370469}

In [186]:
6936617777469473307 in row_id_strange

True

In [193]:
df_diff = bad_integral_metrics_dict['Price']['RowIDHash']

In [171]:
df_diff

,UserID,Price_old,Price_new,Price_diff
0,0,1924,1924.0,True
1,6138819991594274533,21,21.0,True
2,3977008291656862168,155,155.0,True
3,8050505731634518295,1,1.0,True
4,1203066551457721981,200,200.0,True
...,...,...,...,...
206600,7737437551595666831,12,12.0,True
206601,1268123231656295890,1,1.0,True
206602,6324985641656780940,32,32.0,True
206603,7234037711654244978,32,32.0,True


In [3]:
! pip3 install -i https://pypi.yandex-team.ru/simple/ metr_utils --upgrade

Looking in indexes: https://pypi.yandex-team.ru/simple/
     / 32 kB 7.5 MB/ssB/s
ERROR: Could not find a version that satisfies the requirement ipynbname (from metr_utils) (from versions: none)
ERROR: No matching distribution found for ipynbname (from metr_utils)
You should consider upgrading via the '/home/garskova-d/jupyterenv/bin/python3 -m pip install --upgrade pip' command.


In [2]:
! pip3 install https://files.pythonhosted.org/packages/40/be/d3717bd60240255c1f21dd6ba54eb654d7a5590c52f68f12023764f7b073/ipynbname-2021.3.2.tar.gz && pip3 install -i https://pypi.yandex-team.ru/simple/ metr_utils --upgrade --upgrade-strategy only-if-needed

  Created wheel for ipynbname: filename=ipynbname-2021.3.2-py3-none-any.whl size=3200 sha256=715de0af8c48a3d60c35c1849dad4cec7c87a1b2eb581fc3fc8bd989bb7de4bf
  Stored in directory: /home/garskova-d/.cache/pip/wheels/d3/c1/93/6e3ba8770a816a47da1d568b5c4ad4cb5346da9aec6c82bcb3
Successfully built ipynbname
You should consider upgrading via the '/home/garskova-d/jupyterenv/bin/python3 -m pip install --upgrade pip' command.
Looking in indexes: https://pypi.yandex-team.ru/simple/
     - 32 kB 7.0 MB/ssB/s


  Created wheel for metr-utils: filename=metr_utils-2.75-py3-none-any.whl size=45937 sha256=5bf4013c2b5bb9f350c82352b94477e0f5347434e586a355b058e2c617467711
  Stored in directory: /home/garskova-d/.cache/pip/wheels/ed/dd/38/2b8d9df97ff2d810f825389b17368e7e531d7986e08e447c7f
  Created wheel for tabulate: filename=tabulate-0.7.5-py3-none-any.whl size=17959 sha256=732ac0a7bec319fd5fcb85c942938f49513f9fee27d23fd531a5d058c745eb7f
  Stored in directory: /home/garskova-d/.cache/pip/wheels/74/f5/58/2190b36ecc65c45d491eab42280d9650eb10bea70e013ab5d4
Successfully built metr-utils tabulate
  Attempting uninstall: metr-utils
    Found existing installation: metr-utils 2.62
    Uninstalling metr-utils-2.62:
      Successfully uninstalled metr-utils-2.62
  Attempting uninstall: tabulate
    Found existing installation: tabulate 0.8.9
    Uninstalling tabulate-0.8.9:
      Successfully uninstalled tabulate-0.8.9
You should consider upgrading via the '/home/garskova-d/jupyterenv/bin/python3 -m pip ins

In [3]:
from metr_utils.notebook_to_arcadia import push_notebook_to_arcadia
push_notebook_to_arcadia(commit_message='METR-50241: check testing')

METR-50241-testing


FileNotFoundError: [Errno 2] No such file or directory: '/home/garskova-d/arcadia/metrika/analytics/tasks/METR_50241_testing.ipynb'